In [1]:
import gensim
import os
import numpy as np
import nltk
from keras.models import Sequential
from keras.layers.recurrent import LSTM,SimpleRNN

Using TensorFlow backend.


In [2]:
SENTENCE_LENGTH = 20

In [3]:
def read_lines(file_path, separator, name_idx, content_idx, limit = 10000):
    ln = 1
    prev_name = None
    sentences = []
    for line in open(file_path, 'r', encoding="utf-8"):
        #print(ln)
        if ln >= limit:
            break
        ln += 1
        items = line.split(separator)
        name = items[name_idx].lower()
        content = items[content_idx].lower()
        if prev_name != name:
            sentences.append(content.split())
        else:
            #print(prev_name, "spoke again")
            sentences[-1].extend([" "] + content.split() )
        prev_name = name
    return sentences
sentences = read_lines("movie_lines.txt", " +++$+++ ", 3, 4, 10000)

In [4]:
vw_model = gensim.models.Word2Vec(sentences, min_count = 2) #workers = 4

In [5]:
vw_model.save("word2vec_model.txt")

In [6]:
vw_model.most_similar(positive=['big', 'good'], negative=['small'], topn=1)
vw_model.wv.similarity('you', 'sun')

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


0.99668869219706357

In [10]:
def sentence_to_vector(sentences, vw_model, max_length, padding_char =" "):
    padding = vw_model[padding_char]
    l = len(sentences)
    x_sentences = [sentences[i] for i in range(l) if i % 2 == 0]
    y_sentences = [sentences[i] for i in range(l) if i % 2 == 1]
    x_vec = []
    y_vec = []
    # cache padding
    padding_cache = {}
    for i in range(0, max_length + 1):
        paddings = []
        for j in range(i):
            paddings.append(padding)
        padding_cache[i] = paddings
    for sen in x_sentences:
        vec = [vw_model[x] for x in sen if x in vw_model.wv.vocab][:20]
        vec_len = len(vec)
        x_vec.append(vec +  padding_cache[max_length - vec_len] )
    for sen in y_sentences:
        vec = [vw_model[x] for x in sen if x in vw_model.wv.vocab][:20]
        vec_len = len(vec)
        y_vec.append(vec +  padding_cache[max_length - vec_len] )
    return x_vec, y_vec
x_vec, y_vec = sentence_to_vector(sentences, vw_model, SENTENCE_LENGTH, padding_char = " ")

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [11]:
x_vec = np.array(x_vec,dtype=np.float64)
y_vec = np.array(y_vec,dtype=np.float64) 

In [12]:
x_vec_len = len(x_vec)
y_vec_len = len(y_vec)
x_train = x_vec[:int(x_vec_len * 0.8)]
y_train = x_vec[:int(y_vec_len * 0.8)]
x_test = x_vec[:int(x_vec_len * 0.2)]
y_test = x_vec[:int(y_vec_len * 0.2)]

In [13]:

x_train.shape

(3522, 20, 100)

In [14]:
model = Sequential()
model.add(LSTM(output_dim=100,input_shape=x_train.shape[1:],return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
model.add(LSTM(output_dim=100,input_shape=x_train.shape[1:],return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
model.add(LSTM(output_dim=100,input_shape=x_train.shape[1:],return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
model.add(LSTM(output_dim=100,input_shape=x_train.shape[1:],return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
model.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(recurrent_initializer="glorot_normal", input_shape=(20, 100), kernel_initializer="glorot_normal", units=100, activation="sigmoid", return_sequences=True)`
  from ipykernel import kernelapp as app
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(recurrent_initializer="glorot_normal", input_shape=(20, 100), kernel_initializer="glorot_normal", units=100, activation="sigmoid", return_sequences=True)`
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(recurrent_initializer="glorot_normal", input_shape=(20, 100), kernel_initializer="glorot_normal", units=100, activation="si

In [16]:
model.fit(x_train, y_train, nb_epoch=10,validation_data=(x_test, y_test))
model.save('LSTM500.h5');         

Train on 3522 samples, validate on 880 samples
Epoch 1/10
  32/3522 [..............................] - ETA: 15s - loss: -0.7522 - acc: 0.6000

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


3522/3522 [==============================] - 14s 4ms/step - loss: -0.7549 - acc: 0.6758 - val_loss: -0.7573 - val_acc: 0.7031
Epoch 2/10
3522/3522 [==============================] - 14s 4ms/step - loss: -0.7575 - acc: 0.7564 - val_loss: -0.7581 - val_acc: 0.7934
Epoch 3/10
3522/3522 [==============================] - 14s 4ms/step - loss: -0.7582 - acc: 0.8244 - val_loss: -0.7586 - val_acc: 0.8433
Epoch 4/10
3522/3522 [==============================] - 14s 4ms/step - loss: -0.7586 - acc: 0.9209 - val_loss: -0.7590 - val_acc: 0.9929
Epoch 5/10
3522/3522 [==============================] - 15s 4ms/step - loss: -0.7590 - acc: 0.9927 - val_loss: -0.7593 - val_acc: 0.9929
Epoch 6/10
3522/3522 [==============================] - 15s 4ms/step - loss: -0.7592 - acc: 0.9927 - val_loss: -0.7594 - val_acc: 0.9929
Epoch 7/10
3522/3522 [==============================] - 14s 4ms/step - loss: -0.7593 - acc: 0.9927 - val_loss: -0.7595 - val_acc: 0.9929
Epoch 8/10
3522/3522 [==============================

ImportError: `save_model` requires h5py.

In [28]:
q = "hi"
q = q.split()
padding = vw_model[" "]
padding_cache = {}
for i in range(0, SENTENCE_LENGTH + 1):
    paddings = []
    for j in range(i):
        paddings.append(padding)
    padding_cache[i] = paddings

vec = [vw_model[x] for x in q if x in vw_model.wv.vocab][:20]
vec_len = len(vec)
vec = np.array([vec +  padding_cache[SENTENCE_LENGTH - vec_len] ])

res_vec = model.predict(vec) 

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [27]:
dir(vw_model)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__ignoreds',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__numpys',
 '__recursive_saveloads',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__scipys',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_job',
 '_get_job_params',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_minimize_model',
 '_raw_word_count',
 '_save_specials',
 '_set_train_params',
 '_smart_save',
 '_train_epoch',
 '_update_job_params',
 '_worker_loop',
 'accuracy',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'callbacks',
 'cbow_mean',
 'clear_sims',
 'c

In [37]:
print(res_vec.shape)
[vw_model.most_similar(vec) for vec in res_vec]

(1, 20, 100)


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  from ipykernel import kernelapp as app


[[('executions', 0.7963604927062988),
  ('honest.', 0.7920787930488586),
  ('is?', 0.7834762930870056),
  ('welfare', 0.7831843495368958),
  ('guy...', 0.7810665965080261),
  ('exist,', 0.7804214954376221),
  ('bodies', 0.7785512208938599),
  ('fish', 0.7784492373466492),
  ('bet.', 0.7781290411949158),
  ('shell', 0.7779070138931274)]]